**Dataset**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("enalis/tomatoes-dataset")

print("Path to dataset files:", path)

: 

In [ ]:
import os

dataset_dir = os.path.join(path, "content","ieee-mbl-cls","train")
print("Kategori",os.listdir(dataset_dir))


**Data Preprocessing dan Augmentation**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    shear_range=0.1,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    subset='training'
)

# Generator validation
val_gen = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)



# Coba cek batch
x, y = next(iter(train_gen))
print("Train batch shape:", x.shape)
print("Label batch shape:", y.shape)


In [ ]:
print(train_gen.samples, "train images")
print(val_gen.samples, "validation images")


In [ ]:
# import os
# from PIL import Image

# for folder in os.listdir(dataset_dir):
#     subdir = os.path.join(dataset_dir, folder)
#     if os.path.isdir(subdir):
#         for file in os.listdir(subdir):
#             if file.lower().endswith(('.png', '.jpg', '.jpeg')):
#                 path = os.path.join(subdir, file)
#                 try:
#                     img = Image.open(path)
#                     if img.mode == 'RGBA':
#                         img = img.convert('RGB')
#                         img.save(path)
#                         print(f"Converted {file} in {folder} to RGB ✅")
#                 except Exception as e:
#                     print(f"⚠️ Error on {file}: {e}")

**Feature Extraction**

In [ ]:
def extract_features(model, preprocess_func, generator):
    features = []
    labels = []

    for i, (batch_x, batch_y) in enumerate(generator):
        if len(batch_x) == 0:
            continue
        batch_x = preprocess_func(batch_x)
        feat = model.predict(batch_x, verbose=0)
        features.append(feat)
        labels.append(batch_y)

        # hentikan kalau sudah cukup data
        if hasattr(generator, 'n') and len(features) * generator.batch_size >= generator.n:
            break

    if len(features) == 0:
        raise ValueError("Generator tidak menghasilkan batch apapun!")

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

# Membuka layer terakhir untuk fine-tuning
for layer in model.layers[-20:]:
    layer.trainable = True




In [ ]:
import tensorflow as tf, shutil, os

cache_dir = os.path.expanduser('~/.keras/models')
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)

print("✅ Cache EfficientNet dibersihkan. Jalankan ulang model.")


**Transfer Learning**

In [ ]:
from tensorflow.keras.applications import MobileNetV2, ResNet50, EfficientNetB0
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficient_preprocess
import numpy as np
import time
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


backbones = {
    "MobileNetV2": (
        MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(224,224,3)), 
        mobilenet_preprocess
    ),
    "ResNet50": (
        ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224,224,3)), 
        resnet_preprocess
    )

}


results = []

for name, (model, preprocess_func) in backbones.items():
    print(f"\n🔹 Ekstraksi fitur menggunakan {name}...")
    X_train, y_train = extract_features(model, preprocess_func, train_gen)
    X_val, y_val = extract_features(model, preprocess_func, val_gen)

    # Flatten label one-hot ke argmax
    y_train = np.argmax(y_train, axis=1)
    y_val = np.argmax(y_val, axis=1)

    classifiers = {
        "SVM": SVC(kernel='linear'),
        "KNN": KNeighborsClassifier(n_neighbors=5),
        "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42)
    }

    for clf_name, clf in classifiers.items():
        print(f"   🧠 Training {clf_name} untuk {name}...")
        start = time.time()
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_val)
        acc = accuracy_score(y_val, y_pred)
        elapsed = time.time() - start
        results.append([name, clf_name, acc, elapsed])
        print(f"   ✅ Akurasi: {acc:.4f} | Waktu: {elapsed:.2f} detik")

In [ ]:
import pandas as pd
df_results = pd.DataFrame(results, columns=["Backbone", "Classifier", "Accuracy", "Training Time (s)"])
print("\n📊 Hasil Perbandingan:")
print(df_results.sort_values(by="Accuracy", ascending=False))

**Model Pembanding**